# Depencencies

In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dateutil.relativedelta import relativedelta

# Read Data

In [2]:
PATH = '../data/raw/MunicipiosSequia.xlsx'
data = pd.read_excel(PATH)

In [4]:
# Data processing
def create_index_for_row_data(df, date_col):
    df['index'] = df[date_col].dt.date.astype('str') + '__' + df.CVE_CONCATENADA.astype('str')
    df.set_index('index', inplace=True)
    return df

def transform_numerical_drought_index(idx):
    if isinstance(idx, str):
        return float(idx.replace('D', ''))
    return 0

def row_data_with_index(df):
    target = 'DROUGHT_INDEX'
    id_columns = [col for col in df.columns if isinstance(col, str)]
    data = pd.melt(df, id_vars=id_columns, var_name='DATE', value_name=target)
    data['NEW_DATE'] = data['DATE'].apply(change_high_dates_to28)
    data = create_index_for_row_data(data, date_col='NEW_DATE')
    data[target] = data[target].apply(transform_numerical_drought_index)
    return data

def change_high_dates_to28(val):
    if val.day > 15:
        timechange = '-'.join(str(val.date()).split('-')[:2] + ['28'])
        return pd.to_datetime(timechange)
    return val

# Some features
def get_mean(vals):
    return np.nanmean(vals)
    
def get_std(vals):
    vals = vals[:-2]
    return np.nanstd(vals)

def get_max(vals):
    return np.nanmax(vals)

def get_min(vals):
    return np.nanmin(vals)

def get_median(vals):
    return np.nanmedian(vals)

def get_kurtosis(vals):
    return vals.dropna().kurtosis()

def get_skewness(vals):
    return vals.dropna().skew()

def get_mode(vals):
    return vals.dropna().mode().values[0]

def get_mean_vs_median(vals):
    return get_mean(vals) - get_median(vals)

def get_mean_vs_mode(vals):
    return get_mean(vals) - get_mode(vals)

def get_median_vs_mode(vals):
    return get_median(vals) - get_mean(vals)

In [9]:
def calculate_rolling_window_features(df, days=150):
    target_value = 'DROUGHT_INDEX'
    rolling_df = df.rolling(f'{days}D',on='DATE', min_periods=days/30, closed='left')[target_value]
    df[f'{target_value}__mean__last{days}_days'] = rolling_df.apply(get_mean)
    df[f'{target_value}__std__last{days}_days'] = rolling_df.apply(get_std)
    df[f'{target_value}__max__last{days}_days'] = rolling_df.apply(get_max)
    df[f'{target_value}__min__last{days}_days'] = rolling_df.apply(get_min)
    df[f'{target_value}__median__last{days}_days'] = rolling_df.apply(get_median)
    df[f'{target_value}__kurtosis__last{days}_days'] = rolling_df.apply(get_kurtosis)
    df[f'{target_value}__skewness__last{days}_days'] = rolling_df.apply(get_skewness)
    df[f'{target_value}__mode__last{days}_days'] = rolling_df.apply(get_mode)
    df[f'{target_value}__mean_vs_median__last{days}_days'] = rolling_df.apply(get_mean_vs_median)
    df[f'{target_value}__mean_vs_mode__last{days}_days'] = rolling_df.apply(get_mean_vs_mode)
    df[f'{target_value}__median_vs_mode__last{days}_days'] = rolling_df.apply(get_median_vs_mode)
    features_cols = [col for col in df.columns if '__' in col]
    return df[features_cols]

def calculate_rolling_window_features(df, days, date_col='DATE'):
    target_value = 'DROUGHT_INDEX'
    rolling_df = df.rolling(f'{days}D', on=date_col, min_periods=days//30, closed='left')
    rolling_df_target = rolling_df[target_value]

    df[f'{target_value}__mean__last{days}_days'] = rolling_df_target.mean()
    df[f'{target_value}__std__last{days}_days'] = rolling_df_target.std()
    df[f'{target_value}__max__last{days}_days'] = rolling_df_target.max()
    df[f'{target_value}__min__last{days}_days'] = rolling_df_target.min()
    df[f'{target_value}__median__last{days}_days'] = rolling_df_target.median()
    df[f'{target_value}__kurtosis__last{days}_days'] = rolling_df_target.kurt()
    df[f'{target_value}__skewness__last{days}_days'] = rolling_df_target.skew()
    df[f'{target_value}__range__last{days}_days'] = (
            df[f'{target_value}__max__last{days}_days'] -
            df[f'{target_value}__min__last{days}_days']
    )
    df[f'{target_value}__mean_vs_median__last{days}_days'] = (
            df[f'{target_value}__mean__last{days}_days'] -
            df[f'{target_value}__median__last{days}_days']
    )
    features_cols = [col for col in df.columns if '__' in col]
    return df[features_cols]

In [6]:
drought_data = row_data_with_index(data)

In [25]:
def window_feature_generator(*window):
    drought_data_rolling_list = []
    for w in window:
        drought_data_rolling = drought_data.groupby('CVE_CONCATENADA').apply(calculate_rolling_window_features, days=w)
        drought_data_rolling = drought_data_rolling.reset_index().set_index('index').drop('CVE_CONCATENADA', axis=1)
        drought_data_rolling_list.append(drought_data_rolling)
    drought_data_rolling__all_features = pd.concat(drought_data_rolling_list, axis=1)
    return drought_data_rolling__all_features

In [26]:
drought_data_features = window_feature_generator(90,180,360)

In [27]:
drought_data_features

,DROUGHT_INDEX__mean__last90_days,DROUGHT_INDEX__std__last90_days,DROUGHT_INDEX__max__last90_days,DROUGHT_INDEX__min__last90_days,DROUGHT_INDEX__median__last90_days,DROUGHT_INDEX__kurtosis__last90_days,DROUGHT_INDEX__skewness__last90_days,DROUGHT_INDEX__range__last90_days,DROUGHT_INDEX__mean_vs_median__last90_days,DROUGHT_INDEX__mean__last180_days,...,DROUGHT_INDEX__mean_vs_median__last180_days,DROUGHT_INDEX__mean__last360_days,DROUGHT_INDEX__std__last360_days,DROUGHT_INDEX__max__last360_days,DROUGHT_INDEX__min__last360_days,DROUGHT_INDEX__median__last360_days,DROUGHT_INDEX__kurtosis__last360_days,DROUGHT_INDEX__skewness__last360_days,DROUGHT_INDEX__range__last360_days,DROUGHT_INDEX__mean_vs_median__last360_days
index,,,,,,,,,,,,,,,,,,,,,
2003-01-28__1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-02-28__1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-03-28__1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-04-28__1001,0.00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-05-28__1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-15__32058,1.00,0.0,1.0,1.0,1.0,-3.0,0.0,0.0,0.00,0.636364,...,-0.363636,0.695652,0.764840,2.0,0.0,1.0,-0.974194,0.600726,2.0,-0.304348
2023-03-28__32058,1.00,0.0,1.0,1.0,1.0,-3.0,0.0,0.0,0.00,0.727273,...,-0.272727,0.652174,0.714060,2.0,0.0,1.0,-0.694579,0.638962,2.0,-0.347826
2023-04-28__32058,1.00,0.0,1.0,1.0,1.0,-3.0,0.0,0.0,0.00,0.900000,...,-0.100000,0.545455,0.595801,2.0,0.0,0.5,-0.524359,0.552648,2.0,0.045455


In [20]:
PATH_FEATURES = '../data/interim/drought_data_features.csv'
drought_data_features.to_csv(PATH_FEATURES)
drought_data_features = pd.read_csv(PATH_FEATURES, index_col=0)

In [7]:
displaced_drought_data = drought_data.copy()
for months in (1,3,6):
    column = f'DISPLACED_DATE__MONTHS{months}'
    target =  f'DROUGHT_INDEX__NEXT_MONTHS{months}'
    displaced_drought_data[column] = displaced_drought_data['NEW_DATE'].apply(lambda x: x+relativedelta(months=-months)) 
    test = create_index_for_row_data(displaced_drought_data, column).copy()
    test.rename(columns={'DROUGHT_INDEX': target}, inplace=True)
    drought_data_features[target] = test[target]

In [8]:
target_cols = [col for col in drought_data_features.columns if 'DROUGHT_INDEX__NEXT' in col]

In [9]:
PATH_TARGETS = '../data/interim/drought_data_targets.csv'
drought_data_features[target_cols].to_csv(PATH_TARGETS)

In [10]:
PATH_GENERAL_INFO = '../data/interim/drought_data.csv'
drought_data.to_csv(PATH_GENERAL_INFO)